In [5]:
import numpy as np
import pandas as pd
np.set_printoptions(threshold=123456789)

import math, random
from collections import Counter
from itertools import product
from copy import deepcopy

from rdkit import Chem
from rdkit.Chem import AllChem
import os

# file

In [6]:
def read_batch_smi(filename, pwd = '/home/chujunyi/2_Program/2_output_file/3_drug_feature/0_original_data/'):
    with open(pwd + filename) as f:
        batch_smi = f.readlines()
    return [temp.replace('\n','') for temp in batch_smi]

#### original Yamanishi_08 

In [2]:
drug_dict = np.load('/home/chujunyi/2_Program/0_data_sets/NR/drug_smiles_dict.npy', allow_pickle = True).item()
drug_smiles = ''
for smi in drug_dict.values():
    temp = Chem.MolToSmiles(Chem.MolFromSmiles(smi))
    drug_smiles += temp + '\n'
drug_smiles

'CC(C)C(=O)Nc1ccc([N+](=O)[O-])c(C(F)(F)F)c1\nCC=C(C(=CC)c1ccc(O)cc1)c1ccc(O)cc1\nCC12CCC(=O)C=C1CCC1C2C(O)CC2(C)C1CCC2(O)C(=O)CO\nC=C1CCC(O)CC1=CC=C1CCCC2(C)C1CCC2C(C)CCCC(C)C\nCC1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3(F)C(O)CC21C\nC#CC1(O)CCC2C3CCC4=CC(=O)CCC4C3CCC21C\nCC(CCC(=O)O)C1CCC2C3C(O)CC4CC(O)CCC4(C)C3CCC12C\nCC1(C)NC(=O)N(c2ccc([N+](=O)[O-])c(C(F)(F)F)c2)C1=O\nCC(C=CC(C)C(C)(C)O)C1CCC2C(=CC=C3CC(O)CC(O)C3)CCCC21C\nCOC1C=COC2(C)Oc3c(C)c(O)c4c(O)c(c(C=NN5CCN(C)CC5)c(O)c4c3C2=O)NC(=O)C(C)=CC=CC(C)C(O)C(C)C(O)C(C)C(OC(C)=O)C1C\nCC(=O)C1CCC2C3C=CC4=CC(=O)CCC4(C)C3CCC12C\nCC(C)CCCC(C)C1CCC2C3CC=C4CC(O)CCC4(C)C3CCC12C\nCC(=O)C1CCC2C3CC=C4CC(O)CCC4(C)C3CCC12C\nCC(O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C#N)c(C(F)(F)F)c1\nC#CC1(O)CCC2C3CCC4=CC(=O)CCC4C3CCC21CC\nCC12CCC3c4ccc(O)cc4CCC3C1CCC2O\nC=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(C)(C)O)CC(O)CC1O\nCCCc1nc2c(C)cc(-c3nc4ccccc4n3C)cc2n1Cc1ccc(-c2ccccc2C(=O)O)cc1\nCCCC1OC2CC3C4CCC5=CC(=O)C=CC5(C)C4C(O)CC3(C)C2(C(=O)CO)O1\nCS(=O)(=O)O.Cc1ccc(NC(=O)c2ccc

In [3]:
with open('/home/chujunyi/2_Program/0_data_sets/NR/nr_origin_drug_smiles.smi', 'w') as f:
    f.write(drug_smiles)

#### Updated 

In [2]:
# updated_drug_dict = np.load('/home/chujunyi/2_Program/0_data_sets/NR/updated_drug_smiles_dict.npy', allow_pickle = True).item()
# updated_drug_smiles = ''
# updated_drug_ids_smi = []
# i = 0
# for id_, smi in updated_drug_dict.items():
#     updated_drug_smiles += smi + '\n'
#     updated_drug_ids_smi.append([id_,smi])
#     i += 1
# i

580

In [5]:
# with open('/home/chujunyi/2_Program/0_data_sets/NR/nr_updated_drug_smiles.smi', 'w') as f:
#     f.write(updated_drug_smiles)

In [3]:
# nr_updated_id_smi = pd.DataFrame(updated_drug_ids_smi)
# nr_updated_id_smi.columns= ['id', 'smile']
# nr_updated_id_smi.head()

,id,smile
0,D07800,CCCCC(=O)OCC(=O)C1(O)C(C)CC2C3CCC4=CC(=O)C=CC4...
1,D00088,CC12CCC(=O)C=C1CCC1C2C(O)CC2(C)C1CCC2(O)C(=O)CO
2,D06876,CCC(=O)OC1(C(=O)COC(C)=O)CCC2C3CCC4=CC(=O)CCC4...
3,D03911,CCC(=C(c1ccc(OCCN(C)C)cc1)c1cccc(O)c1)c1ccccc1
4,D02174,CC(=O)OCC(=O)C1(O)C(C)CC2C3CCC4=CC(=O)C=CC4(C)...


# ChemDes MFs

In [7]:
os.chdir('/home/chujunyi/2_Program/2_output_file/3_drug_feature/3_MFs')

In [3]:
def read_fps_file(filename, first_line = 0):
    with open(filename, 'r') as f:
        fps = f.readlines()[first_line:]
    fps = [eval(item.replace('\n', '')) for item in fps]

    fps_list = []
    for item in fps:
        if isinstance(item, tuple):
            fps_list.append(sorted(list(item)))
        else:
            fps_list.append([item])

    assert len(fps) == 580
    return fps_list

In [4]:
def read_bitvector_fps(filename, ex = ''):
    fps = pd.read_csv(filename, index_col = 0)
    fps_list =  [[eval(item[len(ex):]) for item in fps.iloc[i,:][fps.iloc[i,:] == 1].index.tolist()] for i in range(fps.shape[0])]
    assert len(fps_list) == 580
    return fps_list

In [5]:
def if_equal(fp1, fp2, datanum = 580):
    num = 0
    for i in range(datanum):
        if fp1[i] == fp2[i]: 
#             print('Common:'+ str(i), end = ',')
            num += 1
    print('相同的总数=',num)
    return

In [6]:
def generate_fps_matrix(fps, shape_1, shape_2):
    mat = np.zeros((shape_1, shape_2))
    for idx, sample_fps in enumerate(fps):
        mat[idx, sample_fps] = 1
    return mat

### Maccs 
http://www.scbdd.com/pybel_desc/fps-maccs/

###### 文件选择 rdkit

In [7]:
def Maccs():
    '''
    PaDEL: 是给的166-bit的向量。
    CDK(×): 该指纹识别器生成166位MACCS密钥。每个功能的SMARTS模式均来自RDKit。但是，鉴于没有正式和明确列出原始键定义，因此此实现的结果可能与其他结果有所不同。
    Pybel(×): 没有125和166位，很神奇……
    RDKit(√): 和排序后的Chemopy相同。
    Chemopy(√): 排序后和RDKit相同。
    '''
    return

In [8]:
padel_maccs_list = read_bitvector_fps('nr_updated_PaDEL_MACCS_fps.csv', ex = 'MACCSFP')
chemopy_maccs_list = read_fps_file('nr_updated_chemopy_MACCS_fps.csv', first_line = 1)
rdkit_maccs_list = read_fps_file('nr_updated_RDKit_MACCS_fps.csv', first_line = 0)
cdk_maccs_list= read_fps_file('nr_updated_CDK_Maccs_fps.csv', first_line = 0)
pybel_maccs_list= read_fps_file('nr_updated_pybel_Maccs_fps.csv', first_line = 0)

In [9]:
assert chemopy_maccs_list == rdkit_maccs_list
if_equal(chemopy_maccs_list, rdkit_maccs_list)#使用这两个！
if_equal(padel_maccs_list, rdkit_maccs_list)
if_equal(pybel_maccs_list, rdkit_maccs_list)

if_equal(cdk_maccs_list, rdkit_maccs_list)
if_equal(cdk_maccs_list, padel_maccs_list)
if_equal(cdk_maccs_list, pybel_maccs_list)
if_equal(padel_maccs_list, pybel_maccs_list)

相同的总数= 580
相同的总数= 1
相同的总数= 375
相同的总数= 0
相同的总数= 0
相同的总数= 0
相同的总数= 2


### PubChem
都试一下

In [10]:
padel_pubchem_list = read_bitvector_fps('nr_updated_PaDEL_PubChem_fps.csv', ex = 'PubchemFP')
cdk_pubchem_list = read_fps_file('nr_updated_CDK_PubChem_fps.csv', first_line = 0)
if_equal(padel_pubchem_list, cdk_pubchem_list)

相同的总数= 2


In [28]:
# num = 0
# for i in range(580):
#     print(len(padel_pubchem_list[i]), len(cdk_pubchem_list[i]))
#     if len(padel_pubchem_list[i]) == len(cdk_pubchem_list[i]): num += 1
# num

In [29]:
# '''
# 在pubchem的search中输入smiles，点击Downloads，找到<PUBCHEM_CACTVS_SUBSKEYS>
# https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/578160/record/SDF/?record_type=2d&response_type=display
# 使用下面网页的代码，得到881-D，但是结果与PaDEL和CDK均不相同……
# https://chem.libretexts.org/Courses/Intercollegiate_Courses/Cheminformatics_OLCC_(2019)/6%3A_Molecular_Similarity/6.4%3A_Python_Assignment

# '''
# from base64 import b64decode

# def PCFP_BitString(pcfp_base64) :

#     pcfp_bitstring = "".join( ["{:08b}".format(x) for x in b64decode( pcfp_base64 )] )[32:913]
#     return pcfp_bitstring
    
# pcfps = 'AAADcfB4OQAAAAAAAAAAAAAAAAAAAYAAAAAwYIAAAAAAAGDAAAAAGwAACAAAD1SggAICCAAABgCIAqDSCAIAAAAgAAAICAFAAEgAEBIAAQQCQAAFgAAIAYPIzPDPgAAAAAAAAADAAAYAADAAAYAADAAAAA=='
# print( len(PCFP_BitString(pcfps)) )
# # print(PCFP_BitString(pcfps))

#### FP2 1024-D

In [13]:
pybel_fp2_list = read_fps_file('nr_updated_pybel_FP2_fps.csv', first_line = 0)

#### Morgan 
https://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints
###### 手动使用rdkit （2.2_v1_multilabel_updated_datasets用了r=4的）
r=2约等于ECFP4

In [10]:
nr_1_100_smi = read_batch_smi('nr_updated_drug_smiles_1_100.smi')
nr_101_200_smi = read_batch_smi('nr_updated_drug_smiles_101_200.smi')
nr_201_300_smi = read_batch_smi('nr_updated_drug_smiles_201_300.smi')
nr_301_400_smi = read_batch_smi('nr_updated_drug_smiles_301_400.smi')
nr_401_500_smi = read_batch_smi('nr_updated_drug_smiles_401_500.smi')
nr_501_580_smi = read_batch_smi('nr_updated_drug_smiles_501_580.smi')
nr_580_smi = nr_1_100_smi + nr_101_200_smi + nr_201_300_smi + nr_301_400_smi + nr_401_500_smi + nr_501_580_smi
assert len(nr_580_smi) == 580
# nr_580_smi

In [11]:
def Morgan():
    '''
    RDKit: 默认参数中未指定radius，所以还是使用RDKit自己生成吧……
    Chemopy: Fingerprints based on the Morgan algorithm.
            很奇怪的数字，应该是还没有经过hash映射。维度高达4294967295？
            详情可见RDKit中的Explaining bits from Morgan Fingerprints。
    '''

In [12]:
rd_mols = [Chem.MolFromSmiles(smiles) for smiles in nr_580_smi]
assert len(rd_mols) == 580

In [23]:
rdkit_morgan_mat = [list(AllChem.GetMorganFingerprintAsBitVect(mol, radius = 2, nBits = 2048)) for mol in rd_mols]
assert len(rdkit_morgan_mat) == len(rd_mols)
assert len(rdkit_morgan_mat[0]) == 2048

In [13]:
rdkit_morgan_mat4 = [list(AllChem.GetMorganFingerprintAsBitVect(mol, radius = 4, nBits = 2048)) for mol in rd_mols]
assert len(rdkit_morgan_mat4) == len(rd_mols)
assert len(rdkit_morgan_mat4[0]) == 2048

#### GraphOnly:Specialized version of the CDK Fingerprinter which does not take bond orders into account.
##### 用CDK吧……是原始的……

In [24]:
def GraphOnly():
    '''
    CDK:Specialized version of the CDK Fingerprinter which does not take bond orders into account.
    PaDel:1024-D; CDK graph only fingerprints; Specialized version of the Fingerprinter which does not take bond orders into account.
    
    '''
    return

In [25]:
cdk_graph_list = read_fps_file('nr_updated_CDK_Graph_fps.csv', first_line = 0)
padel_graph_list = read_bitvector_fps('nr_updated_PaDEL_GraphOnly_fps.csv', ex = 'GraphFP')
if_equal(cdk_graph_list, padel_graph_list)

相同的总数= 0


### save_fp

In [26]:
rdkit_maccs_mat = generate_fps_matrix(rdkit_maccs_list, 580,167)
padel_pubchem_mat = generate_fps_matrix(padel_pubchem_list, 580,881)
cdk_pubchem_mat = generate_fps_matrix(cdk_pubchem_list, 580,881)
pybel_fp2_mat = generate_fps_matrix(pybel_fp2_list, 580,1024)
cdk_graph_mat = generate_fps_matrix(cdk_graph_list, 580,1024)
# rdkit_morgan_mat

In [27]:
np.savez_compressed('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/v1_nr_U_6fps.npz',
                    rdkit_maccs_np = rdkit_maccs_mat,
                    padel_pubchem_np = padel_pubchem_mat,
                    cdk_pubchem_np = cdk_pubchem_mat,
                    pybel_fp2_np = pybel_fp2_mat,
                    cdk_graph_np = cdk_graph_mat,
                    rdkit_morgan_np = rdkit_morgan_mat)

# 处理MFs
#### 1. 所有的MFs都是由/home/chujunyi/2_Program/0_data_sets/NR/nr_updated_drug_smiles.smi生成的sdf文件得到的，该文件称为smi_which_generate_fps。因此要首先将MFs与这个文件直接concat，得到MFs对应的smiles。
#### 2. 找到smi_which_generate_fps中每个smiles对应的drug id，通过与源文件merge得到（/home/chujunyi/2_Program/0_data_sets/NR/updated_drug_smiles_dict.npy）。将这个结果保存到文件/home/chujunyi/2_Program/0_data_sets/NR/nr_updated_drug_id_smiles.csv
#### 3. 将1.和2.的结果merge，得到MF对应的drug id

In [79]:
# drug_dict = np.load('/home/chujunyi/2_Program/0_data_sets/NR/updated_drug_smiles_dict.npy', allow_pickle = True).item()
# drug_id_smiles = []
# for d_id, smi in drug_dict.items():
#     drug_id_smiles.append([d_id, smi])

# drug_id_smiles = pd.DataFrame(drug_id_smiles, columns = ['drug_id', 'smiles'])

In [20]:
drug_smiles_gener_fp = read_batch_smi('nr_updated_drug_smiles.smi', pwd = '/home/chujunyi/2_Program/0_data_sets/NR/')
drug_smiles_gener_fp = pd.DataFrame(drug_smiles_gener_fp, columns = ['smiles'])

In [6]:
# drug_smiles = pd.merge(drug_id_smiles, drug_smiles_gener_fp, on = 'smiles', indicator = False).drop_duplicates().reset_index(drop = True)
# assert drug_smiles.shape == drug_id_smiles.shape
# drug_smiles.to_csv('/home/chujunyi/2_Program/0_data_sets/NR/nr_updated_drug_id_smiles.csv', index = False)

In [21]:
drug_id_smiles = pd.read_csv('/home/chujunyi/2_Program/0_data_sets/NR/nr_updated_drug_id_smiles.csv')
# drug_id_smiles.head()

In [23]:
with np.load('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/v1_nr_U_6fps.npz', allow_pickle = True) as f:
    rdkit_morgan = f['rdkit_morgan_np']
    rdkit_maccs = f['rdkit_maccs_np']
    padel_pubchem = f['padel_pubchem_np']
    cdk_pubchem = f['cdk_pubchem_np']
    pybel_fp2 = f['pybel_fp2_np']
    cdk_graph = f['cdk_graph_np']

In [14]:
rdkit_morgan4 = rdkit_morgan_mat4

### 去除全0和全1列 ；找到每个分子指纹对应的drug id

In [16]:
def filter_01_columns(fps_np, print_ = True):
    fps = pd.DataFrame(fps_np)
    fps_drop0 = fps.loc[:, ~(fps == 0).all()]
    fps_drop1 = fps_drop0.loc[:, ~(fps_drop0 == 0).all()]
    if print_: print('FPs shape: original = {}, filter0 = {}, filter1 = {}'.format(
                     fps.shape, fps_drop0.shape, fps_drop1.shape))
    return fps_drop1

In [17]:
def concat_id_to_fps(fps, smi_which_generate_fps, id_smi):
    if not isinstance(fps, pd.core.frame.DataFrame): fps = pd.DataFrame(fps)
    if not isinstance(smi_which_generate_fps, pd.core.frame.DataFrame): smi_which_generate_fps = pd.DataFrame(smi_which_generate_fps, columns = ['smiles'])
    if not isinstance(id_smi, pd.core.frame.DataFrame): id_smi = pd.DataFrame(id_smi, columns = ['drug_id', 'smiles'])
        
    fp_merge_smi = pd.concat([smi_which_generate_fps, fps], axis = 1)
    id_smi_fp = pd.merge(id_smi, fp_merge_smi, on = 'smiles').drop_duplicates().reset_index(drop = True)
    assert id_smi_fp.shape == (fps.shape[0], fps.shape[1] + 2)
    
    return id_smi_fp

In [18]:
def all_procedure_process_fps(fps_np, smi_which_generate_fps, id_smi):
    fps_filtered = filter_01_columns(fps_np)
    fps_with_id = concat_id_to_fps(fps_filtered, smi_which_generate_fps, id_smi)
    return fps_filtered, fps_with_id

In [27]:
_, rdkit_morgan_with_id = all_procedure_process_fps(rdkit_morgan, drug_smiles_gener_fp, drug_id_smiles)
_, rdkit_maccs_with_id = all_procedure_process_fps(rdkit_maccs, drug_smiles_gener_fp, drug_id_smiles)
_, padel_pubchem_with_id = all_procedure_process_fps(padel_pubchem, drug_smiles_gener_fp, drug_id_smiles)
_, cdk_pubchem_with_id = all_procedure_process_fps(cdk_pubchem, drug_smiles_gener_fp, drug_id_smiles)
_, pybel_fp2_with_id = all_procedure_process_fps(pybel_fp2, drug_smiles_gener_fp, drug_id_smiles)
_, cdk_graph_with_id = all_procedure_process_fps(cdk_graph, drug_smiles_gener_fp, drug_id_smiles)

FPs shape: original = (580, 2048), filter0 = (580, 1730), filter1 = (580, 1730)
FPs shape: original = (580, 167), filter0 = (580, 156), filter1 = (580, 156)
FPs shape: original = (580, 881), filter0 = (580, 558), filter1 = (580, 558)
FPs shape: original = (580, 881), filter0 = (580, 488), filter1 = (580, 488)
FPs shape: original = (580, 1024), filter0 = (580, 988), filter1 = (580, 988)
FPs shape: original = (580, 1024), filter0 = (580, 848), filter1 = (580, 848)


In [22]:
_, rdkit_morgan4_with_id = all_procedure_process_fps(rdkit_morgan4, drug_smiles_gener_fp, drug_id_smiles)

FPs shape: original = (580, 2048), filter0 = (580, 2037), filter1 = (580, 2037)


In [30]:
rdkit_morgan_with_id.to_csv('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/nr_U_rdkit_morgan_with_id.csv', index = False)
rdkit_maccs_with_id.to_csv('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/nr_U_rdkit_maccs_with_id.csv', index = False)
padel_pubchem_with_id.to_csv('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/nr_U_padel_pubchem_with_id.csv', index = False)
cdk_pubchem_with_id.to_csv('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/nr_U_cdk_pubchem_with_id.csv', index = False)
pybel_fp2_with_id.to_csv('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/nr_U_pybel_fp2_with_id.csv', index = False)
cdk_graph_with_id.to_csv('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/nr_U_cdk_graph_with_id.csv', index = False)

In [23]:
rdkit_morgan4_with_id.to_csv('/home/chujunyi/2_Program/2_output_file/3_drug_feature/4_processed_MDs_MFs/nr_U_rdkit_morgan4_with_id.csv', index = False)